In [29]:
# Run setup code
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tltk
tltk.pos_load()
%matplotlib inline

In [30]:
# Path to the preprocessed data
best_processed_path = 'w_review_train.csv'

In [31]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}
print(CHARS_MAP)
print(len(CHARS_MAP))

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '<': 29, '=': 30, '>': 31, '?': 32, '@': 33, 'A': 34, 'B': 35, 'C': 36, 'D': 37, 'E': 38, 'F': 39, 'G': 40, 'H': 41, 'I': 42, 'J': 43, 'K': 44, 'L': 45, 'M': 46, 'N': 47, 'O': 48, 'P': 49, 'Q': 50, 'R': 51, 'S': 52, 'T': 53, 'U': 54, 'V': 55, 'W': 56, 'X': 57, 'Y': 58, 'Z': 59, '[': 60, '\\': 61, ']': 62, '^': 63, '_': 64, 'a': 65, 'b': 66, 'c': 67, 'd': 68, 'e': 69, 'f': 70, 'g': 71, 'h': 72, 'i': 73, 'j': 74, 'k': 75, 'l': 76, 'm': 77, 'n': 78, 'o': 79, 'other': 80, 'p': 81, 'q': 82, 'r': 83, 's': 84, 't': 85, 'u': 86, 'v': 87, 'w': 88, 'x': 89, 'y': 90, 'z': 91, '}': 92, '~': 93, 'ก': 94, 'ข': 95, 'ฃ': 96, 'ค': 97, 'ฅ': 98, 'ฆ': 99, 'ง': 100, 'จ': 101, 'ฉ': 102, 'ช': 103, 'ซ': 104, 'ฌ': 105, 'ญ': 106, 'ฎ': 107, 'ฏ': 108, 'ฐ': 109, 'ฑ'

In [32]:
from keras.models import Model
from keras.layers import Embedding,Dense, Conv1D, Flatten, TimeDistributed, Dropout
from keras.layers import Input, GRU, Bidirectional
from keras.optimizers import Adam
# def get_my_best_model2():
#     input1 = Input(shape=(21,))
#     x = Embedding(178,8)(input1)
#     x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
#     x = TimeDistributed(Dense(5))(x)ฅ
#     x = Flatten()(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     out = Dense(1, activation='sigmoid')(x)
#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                  loss='binary_crossentropy',
#                  metrics=['acc'])          
#     return model

def get_my_best_model():
    input1 = Input(shape=(21,))
    x = Embedding(178,8)(input1)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='binary_crossentropy',
                 metrics=['acc'])          
    return model
    

ModuleNotFoundError: No module named 'keras'

In [7]:
weight_path_my_best_model2='/data/model_best.h5'
my_best_model2 = get_my_best_model()
my_best_model2.load_weights(weight_path_my_best_model2)
#my_best_model2.make_predict_function()
my_best_model2.summary()

NameError: name 'get_my_best_model' is not defined

In [8]:
df = pd.read_csv(best_processed_path, sep=';', header=None)

In [9]:
df

,0,1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


In [13]:
def create_n_gram_df(df, n_pad):
  """
  Given an input dataframe, create a feature dataframe of shifted characters
  Input:
  df: timeseries of size (N)
  n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.
  
  Output:
  dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
  """
  n_pad_2 = int((n_pad - 1)/2)
  for i in range(n_pad_2):
      df['char-{}'.format(i+1)] = df[0].shift(i + 1)
      df['char{}'.format(i+1)] = df[0].shift(-i - 1)
  return df[n_pad_2: -n_pad_2]

In [14]:
word_test = [x for x in df[0][1]]
print(''.join(word_test))

อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา ตัวร้านตั้งอยู่ที่ถนนพุทธมณฑลสาย 3 ไปตาม ถ.ปิ่นเกล้า-นครชัยศรี เมื่อถึงพุทธมณฑลสาย 3 ก็เลี้ยวเข้าไปประมาณ 500 เมตร ร้านอยู่ทางซ้ายมือค่ะ มีคนบอกมาว่าความพิเศษของร้านนี้คือกุ๊กเก่าและเป็นกุ๊กรุ่นสุดท้ายจาก "ฮก ลก ซิ่ว” ภัตตาคารจีนชื่อดังย่านราชประสงค์ ที่เลิกกิจการไปแล้ว ต้องคนที่อายุเลข 5 ขึ้นไปจึงจะเคยกิน ฮก ลก ซิ่ว  จานเด็ดที่มีขายที่นี่แห่งเดียวในเมืองไทยคือ ปลาเต๋าเต้ย 2 ฤดู เป็นสูตรจากมาเลเซีย นอกนั้นก็มี ผัดผักน้ำมันหอย ไก่เบตง เคาหยก ปูทะเลซุปน้ำใสหม้อไฟ เต้าหู้แคระยัดไส้หม้อดิน และ ลูกชิ้นแคระ 
อาหารที่เราแนะนำคือไก่เบตง (คล้ายๆไก่แช่เหล้า) เสริฟพร้อมกับหอมเจียว และน้ำจิ้มน้ำพริกเผาสูตรเด็ดของทางร้าน 
เมนูข้าวผัดหนำเลียบก็อร่อยค่ะ ชอบมากๆ


In [15]:
word_test_df2 = pd.DataFrame(word_test)
#print(word_test_df2)

n_pad = 21
n_pad_2 = int((n_pad - 1)/2)
pad = [{0: ' '}]
df_pad = pd.DataFrame(pad * n_pad_2)
#print(df_pad)

word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
#print(word_test_df2[0][:21])
word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))

In [16]:
df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
             ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

# convert pandas dataframe to numpy array to feed to the model
x_char = df_with_context[char_row].as_matrix()

In [17]:
y_pred = my_best_model2.predict(x_char)

prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
y_pred = np.apply_along_axis(prob_to_class,1,y_pred)

In [18]:
print(len(y_pred))
print(len(word_test))
word_token = []
word_tmp = ''
for i in range(len(y_pred)):
    if(y_pred[i] == 1):
        word_token.append(word_tmp)
        word_tmp = ''
    word_tmp += word_test[i]
else:
    word_token.append(word_tmp)
word_token = word_token[1:]
print(word_token)

701
701
['อาหาร', 'ที่', 'นี่', 'เป็น', 'อาหาร', 'จีน', 'แคะ', 'ที่', 'หากิน', 'ยาก', 'ใน', 'บ้าน', 'เรา', ' ', 'ตัว', 'ร้าน', 'ตั้ง', 'อยู่', 'ที่', 'ถนนพุทธมณฑล', 'สาย', ' ', '3', ' ', 'ไป', 'ตาม', ' ', 'ถ.', 'ปิ่นเกล้า', '-', 'นครชัยศรี', ' ', 'เมื่อ', 'ถึง', 'พุทธมณฑล', 'สาย', ' ', '3', ' ', 'ก็', 'เลี้ยว', 'เข้า', 'ไป', 'ประมาณ', ' ', '500', ' ', 'เมตร', ' ', 'ร้าน', 'อยู่', 'ทาง', 'ซ้าย', 'มือ', 'ค่ะ', ' ', 'มี', 'คน', 'บอก', 'มา', 'ว่า', 'ความ', 'พิเศษ', 'ของ', 'ร้าน', 'นี้', 'คือ', 'กุ๊ก', 'เก่า', 'และ', 'เป็น', 'กุ๊ก', 'รุ่น', 'สุดท้าย', 'จาก', ' ', '"', 'ฮก', ' ', 'ลก', ' ', 'ซิ่ว', '”', ' ', 'ภัตตา', 'คาร', 'จีน', 'ชื่อ', 'ดัง', 'ย่าน', 'ราชประสงค์', ' ', 'ที่', 'เลิก', 'กิจการ', 'ไป', 'แล้ว', ' ', 'ต้อง', 'คน', 'ที่', 'อายุ', 'เลข', ' ', '5', ' ', 'ขึ้น', 'ไป', 'จึง', 'จะ', 'เคย', 'กิน', ' ', 'ฮก', ' ', 'ลก', ' ', 'ซิ่ว', ' ', ' ', 'จาน', 'เด็ด', 'ที่', 'มี', 'ขาย', 'ที่', 'นี่', 'แห่ง', 'เดียว', 'ใน', 'เมืองไทย', 'คือ', ' ', 'ปลาเต๋าเต้ย', ' ', '2', ' ', 'ฤดู', ' ', 'เป็น'

In [19]:
def cutKumKrub(string):
    word_test = [x for x in string]
    word_test_df2 = pd.DataFrame(word_test)

    n_pad = 21
    n_pad_2 = int((n_pad - 1)/2)
    pad = [{0: ' '}]
    df_pad = pd.DataFrame(pad * n_pad_2)

    word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
    word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))
    
    df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

    char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
                 ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

    # convert pandas dataframe to numpy array to feed to the model
    x_char = df_with_context[char_row].as_matrix()
    
    y_pred = my_best_model2.predict(x_char)

    prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
    y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    
    word_token = []
    word_tmp = ''
    for i in range(len(y_pred)):
        if(y_pred[i] == 1):
            word_token.append(word_tmp)
            word_tmp = ''
        word_tmp += word_test[i]
    else:
        word_token.append(word_tmp)
    word_token = word_token[1:]
    return word_token

In [20]:
def mergeLabel(review, rating):
    noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
    cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
    for i in range(len(cleaned)):
        cleaned[i] = cleaned[i].replace("\n", "")
        cleaned[i] = cleaned[i].replace("(", "")
        cleaned[i] = cleaned[i].replace(")", "")
    finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))
    label = [int(rating)] * len(cleaned)
    training_data = list(zip(cleaned, label))
    return training_data

In [21]:
mergeLabel(df[0][1], df[1][1])

[('อาหาร', 4),
 ('ที่', 4),
 ('นี่', 4),
 ('เป็น', 4),
 ('อาหาร', 4),
 ('จีน', 4),
 ('แคะ', 4),
 ('ที่', 4),
 ('หากิน', 4),
 ('ยาก', 4),
 ('ใน', 4),
 ('บ้าน', 4),
 ('เรา', 4),
 ('ตัว', 4),
 ('ร้าน', 4),
 ('ตั้ง', 4),
 ('อยู่', 4),
 ('ที่', 4),
 ('ถนนพุทธมณฑล', 4),
 ('สาย', 4),
 ('3', 4),
 ('ไป', 4),
 ('ตาม', 4),
 ('ถ.', 4),
 ('ปิ่นเกล้า', 4),
 ('-', 4),
 ('นครชัยศรี', 4),
 ('เมื่อ', 4),
 ('ถึง', 4),
 ('พุทธมณฑล', 4),
 ('สาย', 4),
 ('3', 4),
 ('ก็', 4),
 ('เลี้ยว', 4),
 ('เข้า', 4),
 ('ไป', 4),
 ('ประมาณ', 4),
 ('500', 4),
 ('เมตร', 4),
 ('ร้าน', 4),
 ('อยู่', 4),
 ('ทาง', 4),
 ('ซ้าย', 4),
 ('มือ', 4),
 ('ค่ะ', 4),
 ('มี', 4),
 ('คน', 4),
 ('บอก', 4),
 ('มา', 4),
 ('ว่า', 4),
 ('ความ', 4),
 ('พิเศษ', 4),
 ('ของ', 4),
 ('ร้าน', 4),
 ('นี้', 4),
 ('คือ', 4),
 ('กุ๊ก', 4),
 ('เก่า', 4),
 ('และ', 4),
 ('เป็น', 4),
 ('กุ๊ก', 4),
 ('รุ่น', 4),
 ('สุดท้าย', 4),
 ('จาก', 4),
 ('"', 4),
 ('ฮก', 4),
 ('ลก', 4),
 ('ซิ่ว', 4),
 ('”', 4),
 ('ภัตตา', 4),
 ('คาร', 4),
 ('จีน', 4),
 ('ชื่อ', 4),
 ('ดั

In [168]:
demo = df[:][:10000]
demo

,0,1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


In [169]:
train = []
for i in range(len(demo)):
    train.append((demo[0][i], demo[1][i]))

In [170]:
# train

In [174]:
import re
def remove_htmlTags(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext
    
def isValidWord(text):
    dic = {text[0]:1}
    for c in text[1:]:
        if c not in dic:
            return True
        else:
            dic[c] += 1
    if dic[text[0]] > 1:
        return False
    else:
        return True
        
def tokenize(review):
    
#     tags = tltk.nlp.pos_tag(review.replace("\n", ""))
#     noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
#     cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
#     for i in range(len(cleaned)):
#         cleaned[i] = cleaned[i].replace("\n", "")
#         cleaned[i] = cleaned[i].replace("(", "")
#         cleaned[i] = cleaned[i].replace(")", "")
#     finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))

    # Filter parts of speech of text
    tags = tltk.nlp.pos_tag(remove_htmlTags(review.replace("\n", "")))
    finalWord = []
    filteredTags = {'NOUN', 'VERB', 'ADJ','ADV', 'AUX', 'PART'}
    for sentence in tags:
        for word, tag in sentence:
            if tag in filteredTags:
                if len(word) > 1 and isValidWord(word):
                    finalWord.append(remove_htmlTags(word))
    return finalWord

# tokenize(train[0][0])
# train[0][0]

In [175]:
# from itertools import chain
# vocabulary = set(chain(*[tokenize(i[0].lower()) for i in train]))

In [176]:
# vocabulary
tokenize(train[i][0])

['ร้าน',
 'แหลม',
 'เจริญ',
 'ร้าน',
 'อาหาร',
 'ชื่อดัง',
 'ขึ้นชื่อ',
 'เรื่อง',
 'อาหาร',
 'ทะ',
 'เลย',
 'มี',
 'สาขา',
 'ทั่ว',
 'กทม.',
 'สาขา',
 'เหมือน',
 'สาขา',
 'แหละ',
 'ค่ะ',
 'รสชาติ',
 'อาหาร',
 'การบริการ',
 'ดี',
 'ค่ะ',
 'ร้าน',
 'มี',
 'ที่นั่ง',
 'เยอะ',
 'ดี',
 'ลูกค้า',
 'เต็ม',
 'ร้าน',
 'อาหาร',
 'อร่อย',
 'แทบ',
 'ทุก',
 'เมนู',
 'อาหาร',
 'ทะเล',
 'สด',
 'ดี',
 'ค่ะ',
 'ราคา',
 'สมเหตุสมผล']

In [177]:
from IPython.display import clear_output
corpus = []
for i in range(len(train)):
    print(str(i+1) + "/" + str(len(train)))
    corpus.append(tokenize(train[i][0]))

corpus
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

1/10000
2/10000
3/10000
4/10000
5/10000
6/10000
7/10000
8/10000
9/10000
10/10000
11/10000
12/10000
13/10000
14/10000
15/10000
16/10000
17/10000
18/10000
19/10000
20/10000
21/10000
22/10000
23/10000
24/10000
25/10000
26/10000
27/10000
28/10000
29/10000
30/10000
31/10000
32/10000
33/10000
34/10000
35/10000
36/10000
37/10000
38/10000
39/10000
40/10000
41/10000
42/10000
43/10000
44/10000
45/10000
46/10000
47/10000
48/10000
49/10000
50/10000
51/10000
52/10000
53/10000
54/10000
55/10000
56/10000
57/10000
58/10000
59/10000
60/10000
61/10000
62/10000
63/10000
64/10000
65/10000
66/10000
67/10000
68/10000
69/10000
70/10000
71/10000
72/10000
73/10000
74/10000
75/10000
76/10000
77/10000
78/10000
79/10000
80/10000
81/10000
82/10000
83/10000
84/10000
85/10000
86/10000
87/10000
88/10000
89/10000
90/10000
91/10000
92/10000
93/10000
94/10000
95/10000
96/10000
97/10000
98/10000
99/10000
100/10000
101/10000
102/10000
103/10000
104/10000
105/10000
106/10000
107/10000
108/10000
109/10000
110/10000
111/1000

832/10000
833/10000
834/10000
835/10000
836/10000
837/10000
838/10000
839/10000
840/10000
841/10000
842/10000
843/10000
844/10000
845/10000
846/10000
847/10000
848/10000
849/10000
850/10000
851/10000
852/10000
853/10000
854/10000
855/10000
856/10000
857/10000
858/10000
859/10000
860/10000
861/10000
862/10000
863/10000
864/10000
865/10000
866/10000
867/10000
868/10000
869/10000
870/10000
871/10000
872/10000
873/10000
874/10000
875/10000
876/10000
877/10000
878/10000
879/10000
880/10000
881/10000
882/10000
883/10000
884/10000
885/10000
886/10000
887/10000
888/10000
889/10000
890/10000
891/10000
892/10000
893/10000
894/10000
895/10000
896/10000
897/10000
898/10000
899/10000
900/10000
901/10000
902/10000
903/10000
904/10000
905/10000
906/10000
907/10000
908/10000
909/10000
910/10000
911/10000
912/10000
913/10000
914/10000
915/10000
916/10000
917/10000
918/10000
919/10000
920/10000
921/10000
922/10000
923/10000
924/10000
925/10000
926/10000
927/10000
928/10000
929/10000
930/10000
931/10000


1593/10000
1594/10000
1595/10000
1596/10000
1597/10000
1598/10000
1599/10000
1600/10000
1601/10000
1602/10000
1603/10000
1604/10000
1605/10000
1606/10000
1607/10000
1608/10000
1609/10000
1610/10000
1611/10000
1612/10000
1613/10000
1614/10000
1615/10000
1616/10000
1617/10000
1618/10000
1619/10000
1620/10000
1621/10000
1622/10000
1623/10000
1624/10000
1625/10000
1626/10000
1627/10000
1628/10000
1629/10000
1630/10000
1631/10000
1632/10000
1633/10000
1634/10000
1635/10000
1636/10000
1637/10000
1638/10000
1639/10000
1640/10000
1641/10000
1642/10000
1643/10000
1644/10000
1645/10000
1646/10000
1647/10000
1648/10000
1649/10000
1650/10000
1651/10000
1652/10000
1653/10000
1654/10000
1655/10000
1656/10000
1657/10000
1658/10000
1659/10000
1660/10000
1661/10000
1662/10000
1663/10000
1664/10000
1665/10000
1666/10000
1667/10000
1668/10000
1669/10000
1670/10000
1671/10000
1672/10000
1673/10000
1674/10000
1675/10000
1676/10000
1677/10000
1678/10000
1679/10000
1680/10000
1681/10000
1682/10000
1683/10000

2338/10000
2339/10000
2340/10000
2341/10000
2342/10000
2343/10000
2344/10000
2345/10000
2346/10000
2347/10000
2348/10000
2349/10000
2350/10000
2351/10000
2352/10000
2353/10000
2354/10000
2355/10000
2356/10000
2357/10000
2358/10000
2359/10000
2360/10000
2361/10000
2362/10000
2363/10000
2364/10000
2365/10000
2366/10000
2367/10000
2368/10000
2369/10000
2370/10000
2371/10000
2372/10000
2373/10000
2374/10000
2375/10000
2376/10000
2377/10000
2378/10000
2379/10000
2380/10000
2381/10000
2382/10000
2383/10000
2384/10000
2385/10000
2386/10000
2387/10000
2388/10000
2389/10000
2390/10000
2391/10000
2392/10000
2393/10000
2394/10000
2395/10000
2396/10000
2397/10000
2398/10000
2399/10000
2400/10000
2401/10000
2402/10000
2403/10000
2404/10000
2405/10000
2406/10000
2407/10000
2408/10000
2409/10000
2410/10000
2411/10000
2412/10000
2413/10000
2414/10000
2415/10000
2416/10000
2417/10000
2418/10000
2419/10000
2420/10000
2421/10000
2422/10000
2423/10000
2424/10000
2425/10000
2426/10000
2427/10000
2428/10000

3083/10000
3084/10000
3085/10000
3086/10000
3087/10000
3088/10000
3089/10000
3090/10000
3091/10000
3092/10000
3093/10000
3094/10000
3095/10000
3096/10000
3097/10000
3098/10000
3099/10000
3100/10000
3101/10000
3102/10000
3103/10000
3104/10000
3105/10000
3106/10000
3107/10000
3108/10000
3109/10000
3110/10000
3111/10000
3112/10000
3113/10000
3114/10000
3115/10000
3116/10000
3117/10000
3118/10000
3119/10000
3120/10000
3121/10000
3122/10000
3123/10000
3124/10000
3125/10000
3126/10000
3127/10000
3128/10000
3129/10000
3130/10000
3131/10000
3132/10000
3133/10000
3134/10000
3135/10000
3136/10000
3137/10000
3138/10000
3139/10000
3140/10000
3141/10000
3142/10000
3143/10000
3144/10000
3145/10000
3146/10000
3147/10000
3148/10000
3149/10000
3150/10000
3151/10000
3152/10000
3153/10000
3154/10000
3155/10000
3156/10000
3157/10000
3158/10000
3159/10000
3160/10000
3161/10000
3162/10000
3163/10000
3164/10000
3165/10000
3166/10000
3167/10000
3168/10000
3169/10000
3170/10000
3171/10000
3172/10000
3173/10000

3828/10000
3829/10000
3830/10000
3831/10000
3832/10000
3833/10000
3834/10000
3835/10000
3836/10000
3837/10000
3838/10000
3839/10000
3840/10000
3841/10000
3842/10000
3843/10000
3844/10000
3845/10000
3846/10000
3847/10000
3848/10000
3849/10000
3850/10000
3851/10000
3852/10000
3853/10000
3854/10000
3855/10000
3856/10000
3857/10000
3858/10000
3859/10000
3860/10000
3861/10000
3862/10000
3863/10000
3864/10000
3865/10000
3866/10000
3867/10000
3868/10000
3869/10000
3870/10000
3871/10000
3872/10000
3873/10000
3874/10000
3875/10000
3876/10000
3877/10000
3878/10000
3879/10000
3880/10000
3881/10000
3882/10000
3883/10000
3884/10000
3885/10000
3886/10000
3887/10000
3888/10000
3889/10000
3890/10000
3891/10000
3892/10000
3893/10000
3894/10000
3895/10000
3896/10000
3897/10000
3898/10000
3899/10000
3900/10000
3901/10000
3902/10000
3903/10000
3904/10000
3905/10000
3906/10000
3907/10000
3908/10000
3909/10000
3910/10000
3911/10000
3912/10000
3913/10000
3914/10000
3915/10000
3916/10000
3917/10000
3918/10000

4573/10000
4574/10000
4575/10000
4576/10000
4577/10000
4578/10000
4579/10000
4580/10000
4581/10000
4582/10000
4583/10000
4584/10000
4585/10000
4586/10000
4587/10000
4588/10000
4589/10000
4590/10000
4591/10000
4592/10000
4593/10000
4594/10000
4595/10000
4596/10000
4597/10000
4598/10000
4599/10000
4600/10000
4601/10000
4602/10000
4603/10000
4604/10000
4605/10000
4606/10000
4607/10000
4608/10000
4609/10000
4610/10000
4611/10000
4612/10000
4613/10000
4614/10000
4615/10000
4616/10000
4617/10000
4618/10000
4619/10000
4620/10000
4621/10000
4622/10000
4623/10000
4624/10000
4625/10000
4626/10000
4627/10000
4628/10000
4629/10000
4630/10000
4631/10000
4632/10000
4633/10000
4634/10000
4635/10000
4636/10000
4637/10000
4638/10000
4639/10000
4640/10000
4641/10000
4642/10000
4643/10000
4644/10000
4645/10000
4646/10000
4647/10000
4648/10000
4649/10000
4650/10000
4651/10000
4652/10000
4653/10000
4654/10000
4655/10000
4656/10000
4657/10000
4658/10000
4659/10000
4660/10000
4661/10000
4662/10000
4663/10000

5318/10000
5319/10000
5320/10000
5321/10000
5322/10000
5323/10000
5324/10000
5325/10000
5326/10000
5327/10000
5328/10000
5329/10000
5330/10000
5331/10000
5332/10000
5333/10000
5334/10000
5335/10000
5336/10000
5337/10000
5338/10000
5339/10000
5340/10000
5341/10000
5342/10000
5343/10000
5344/10000
5345/10000
5346/10000
5347/10000
5348/10000
5349/10000
5350/10000
5351/10000
5352/10000
5353/10000
5354/10000
5355/10000
5356/10000
5357/10000
5358/10000
5359/10000
5360/10000
5361/10000
5362/10000
5363/10000
5364/10000
5365/10000
5366/10000
5367/10000
5368/10000
5369/10000
5370/10000
5371/10000
5372/10000
5373/10000
5374/10000
5375/10000
5376/10000
5377/10000
5378/10000
5379/10000
5380/10000
5381/10000
5382/10000
5383/10000
5384/10000
5385/10000
5386/10000
5387/10000
5388/10000
5389/10000
5390/10000
5391/10000
5392/10000
5393/10000
5394/10000
5395/10000
5396/10000
5397/10000
5398/10000
5399/10000
5400/10000
5401/10000
5402/10000
5403/10000
5404/10000
5405/10000
5406/10000
5407/10000
5408/10000

IndexError: string index out of range

In [182]:
print(len(corpus))
import pickle


pickle_out = open("corpus.pickle","wb")
pickle.dump(corpus, pickle_out)
pickle_out.close()


5876


In [191]:
corpus2 = []
for i in range(5876, len(train)):
    print(str(i+1) + "/" + str(len(train)))
    corpus2.append(tokenize(train[i][0].replace("|", "")))

corpus2

5877/10000
5878/10000
5879/10000
5880/10000
5881/10000
5882/10000
5883/10000
5884/10000
5885/10000
5886/10000
5887/10000
5888/10000
5889/10000
5890/10000
5891/10000
5892/10000
5893/10000
5894/10000
5895/10000
5896/10000
5897/10000
5898/10000
5899/10000
5900/10000
5901/10000
5902/10000
5903/10000
5904/10000
5905/10000
5906/10000
5907/10000
5908/10000
5909/10000
5910/10000
5911/10000
5912/10000
5913/10000
5914/10000
5915/10000
5916/10000
5917/10000
5918/10000
5919/10000
5920/10000
5921/10000
5922/10000
5923/10000
5924/10000
5925/10000
5926/10000
5927/10000
5928/10000
5929/10000
5930/10000
5931/10000
5932/10000
5933/10000
5934/10000
5935/10000
5936/10000
5937/10000
5938/10000
5939/10000
5940/10000
5941/10000
5942/10000
5943/10000
5944/10000
5945/10000
5946/10000
5947/10000
5948/10000
5949/10000
5950/10000
5951/10000
5952/10000
5953/10000
5954/10000
5955/10000
5956/10000
5957/10000
5958/10000
5959/10000
5960/10000
5961/10000
5962/10000
5963/10000
5964/10000
5965/10000
5966/10000
5967/10000

6622/10000
6623/10000
6624/10000
6625/10000
6626/10000
6627/10000
6628/10000
6629/10000
6630/10000
6631/10000
6632/10000
6633/10000
6634/10000
6635/10000
6636/10000
6637/10000
6638/10000
6639/10000
6640/10000
6641/10000
6642/10000
6643/10000
6644/10000
6645/10000
6646/10000
6647/10000
6648/10000
6649/10000
6650/10000
6651/10000
6652/10000
6653/10000
6654/10000
6655/10000
6656/10000
6657/10000
6658/10000
6659/10000
6660/10000
6661/10000
6662/10000
6663/10000
6664/10000
6665/10000
6666/10000
6667/10000
6668/10000
6669/10000
6670/10000
6671/10000
6672/10000
6673/10000
6674/10000
6675/10000
6676/10000
6677/10000
6678/10000
6679/10000
6680/10000
6681/10000
6682/10000
6683/10000
6684/10000
6685/10000
6686/10000
6687/10000
6688/10000
6689/10000
6690/10000
6691/10000
6692/10000
6693/10000
6694/10000
6695/10000
6696/10000
6697/10000
6698/10000
6699/10000
6700/10000
6701/10000
6702/10000
6703/10000
6704/10000
6705/10000
6706/10000
6707/10000
6708/10000
6709/10000
6710/10000
6711/10000
6712/10000

7367/10000
7368/10000
7369/10000
7370/10000
7371/10000
7372/10000
7373/10000
7374/10000
7375/10000
7376/10000
7377/10000
7378/10000
7379/10000
7380/10000
7381/10000
7382/10000
7383/10000
7384/10000
7385/10000
7386/10000
7387/10000
7388/10000
7389/10000
7390/10000
7391/10000
7392/10000
7393/10000
7394/10000
7395/10000
7396/10000
7397/10000
7398/10000
7399/10000
7400/10000
7401/10000
7402/10000
7403/10000
7404/10000
7405/10000
7406/10000
7407/10000
7408/10000
7409/10000
7410/10000
7411/10000
7412/10000
7413/10000
7414/10000
7415/10000
7416/10000
7417/10000
7418/10000
7419/10000
7420/10000
7421/10000
7422/10000
7423/10000
7424/10000
7425/10000
7426/10000
7427/10000
7428/10000
7429/10000
7430/10000
7431/10000
7432/10000
7433/10000
7434/10000
7435/10000
7436/10000
7437/10000
7438/10000
7439/10000
7440/10000
7441/10000
7442/10000
7443/10000
7444/10000
7445/10000
7446/10000
7447/10000
7448/10000
7449/10000
7450/10000
7451/10000
7452/10000
7453/10000
7454/10000
7455/10000
7456/10000
7457/10000

8112/10000
8113/10000
8114/10000
8115/10000
8116/10000
8117/10000
8118/10000
8119/10000
8120/10000
8121/10000
8122/10000
8123/10000
8124/10000
8125/10000
8126/10000
8127/10000
8128/10000
8129/10000
8130/10000
8131/10000
8132/10000
8133/10000
8134/10000
8135/10000
8136/10000
8137/10000
8138/10000
8139/10000
8140/10000
8141/10000
8142/10000
8143/10000
8144/10000
8145/10000
8146/10000
8147/10000
8148/10000
8149/10000
8150/10000
8151/10000
8152/10000
8153/10000
8154/10000
8155/10000
8156/10000
8157/10000
8158/10000
8159/10000
8160/10000
8161/10000
8162/10000
8163/10000
8164/10000
8165/10000
8166/10000
8167/10000
8168/10000
8169/10000
8170/10000
8171/10000
8172/10000
8173/10000
8174/10000
8175/10000
8176/10000
8177/10000
8178/10000
8179/10000
8180/10000
8181/10000
8182/10000
8183/10000
8184/10000
8185/10000
8186/10000
8187/10000
8188/10000
8189/10000
8190/10000
8191/10000
8192/10000
8193/10000
8194/10000
8195/10000
8196/10000
8197/10000
8198/10000
8199/10000
8200/10000
8201/10000
8202/10000

8857/10000
8858/10000
8859/10000
8860/10000
8861/10000
8862/10000
8863/10000
8864/10000
8865/10000
8866/10000
8867/10000
8868/10000
8869/10000
8870/10000
8871/10000
8872/10000
8873/10000
8874/10000
8875/10000
8876/10000
8877/10000
8878/10000
8879/10000
8880/10000
8881/10000
8882/10000
8883/10000
8884/10000
8885/10000
8886/10000
8887/10000
8888/10000
8889/10000
8890/10000
8891/10000
8892/10000
8893/10000
8894/10000
8895/10000
8896/10000
8897/10000
8898/10000
8899/10000
8900/10000
8901/10000
8902/10000
8903/10000
8904/10000
8905/10000
8906/10000
8907/10000
8908/10000
8909/10000
8910/10000
8911/10000
8912/10000
8913/10000
8914/10000
8915/10000
8916/10000
8917/10000
8918/10000
8919/10000
8920/10000
8921/10000
8922/10000
8923/10000
8924/10000
8925/10000
8926/10000
8927/10000
8928/10000
8929/10000
8930/10000
8931/10000
8932/10000
8933/10000
8934/10000
8935/10000
8936/10000
8937/10000
8938/10000
8939/10000
8940/10000
8941/10000
8942/10000
8943/10000
8944/10000
8945/10000
8946/10000
8947/10000

9602/10000
9603/10000
9604/10000
9605/10000
9606/10000
9607/10000
9608/10000
9609/10000
9610/10000
9611/10000
9612/10000
9613/10000
9614/10000
9615/10000
9616/10000
9617/10000
9618/10000
9619/10000
9620/10000
9621/10000
9622/10000
9623/10000
9624/10000
9625/10000
9626/10000
9627/10000
9628/10000
9629/10000
9630/10000
9631/10000
9632/10000
9633/10000
9634/10000
9635/10000
9636/10000
9637/10000
9638/10000
9639/10000
9640/10000
9641/10000
9642/10000
9643/10000
9644/10000
9645/10000
9646/10000
9647/10000
9648/10000
9649/10000
9650/10000
9651/10000
9652/10000
9653/10000
9654/10000
9655/10000
9656/10000
9657/10000
9658/10000
9659/10000
9660/10000
9661/10000
9662/10000
9663/10000
9664/10000
9665/10000
9666/10000
9667/10000
9668/10000
9669/10000
9670/10000
9671/10000
9672/10000
9673/10000
9674/10000
9675/10000
9676/10000
9677/10000
9678/10000
9679/10000
9680/10000
9681/10000
9682/10000
9683/10000
9684/10000
9685/10000
9686/10000
9687/10000
9688/10000
9689/10000
9690/10000
9691/10000
9692/10000

[['ได้',
  'ลอง',
  'มา',
  'ทาน',
  'อาหาร',
  'อีสาน',
  'ร้าน',
  'ค่ะ',
  'บรรยากาศ',
  'ร้าน',
  'มี',
  'ที่จอดรถ',
  'กว้างขวาง',
  'ดี',
  'ค่ะ',
  'หน้าร้าน',
  'ฝุ่น',
  'เยอะ',
  'ไป',
  'นิด',
  'เป็น',
  'พื้นดิน',
  'รวม',
  'แล้ว',
  'ให้',
  'ผ่าน',
  'ค่ะ',
  'ร้าน',
  'สะอาด',
  'อุปกรณ์จานช้ิน',
  'ทุก',
  'อย่าง',
  'จะ',
  'ซิล',
  'พลาสติก',
  'หมด',
  'จาน',
  'ใส่',
  'ผัก',
  'สด',
  'ยัง',
  'ซิล',
  'มา',
  'อย่าง',
  'ดี',
  'คง',
  'ป้องกัน',
  'ฝุ่น',
  'มา',
  'รสชาติ',
  'อาหาร',
  'ชอบ',
  'รส',
  'จัด',
  'ต้อง',
  'ชอบ',
  'แน่',
  'ส้มตำ',
  'ต้มแซบ',
  'ยำ',
  'อย่า',
  'พลาด',
  'น้า',
  'เมนู',
  'แนะนำ',
  'อร่อย',
  'ค่ะ',
  'อัน',
  'เต็ม',
  'เลย'],
 ['อยาก',
  'ลอง',
  'ใหม่',
  'โทส',
  'ขนมปัง',
  'สี',
  'ดำ',
  'ราด',
  'ด้วย',
  'ซอส',
  'ถั่ว',
  'คู่',
  'ไอ',
  'ครีม',
  'วนิ',
  'ลา',
  'รสชาติ',
  'ครั้ง',
  'แรก',
  'ทาน',
  'อร่อย',
  'คำ',
  'จะ',
  'รู้สึก',
  'เลี่ยน',
  'มาก',
  'ตัว',
  'ขนมปัง',
  'จะ',
  'แน่น',
  'มาก',
  

In [192]:

import pickle


pickle_out = open("corpus2.pickle","wb")
pickle.dump(corpus2, pickle_out)
pickle_out.close()



In [44]:
# pickle_in = open("corpus.pickle","rb")
# corpus1 = pickle.load(pickle_in)
    
# corpus1

In [69]:
corpus
newcor = corpus[:250]
test = corpus[250:]
# tltk.nlp.pos_tag(train[0][0])

In [163]:
import gensim
from collections import defaultdict
# let X be a list of tokenized texts (i.e. list of lists of tokens)
model = gensim.models.Word2Vec(newcor, size=250)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        for tmp in word2vec:
            self.dim = len(word2vec[tmp])
            break

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        m = tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return m

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

/Users/fm_index/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """


In [164]:
y_train = []

for i in range(250):
    y_train.append(train[i][1])
y_train = np.array(y_train)

tfidf = TfidfEmbeddingVectorizer(w2v)
tfidf_matrix = tfidf.fit(newcor, y_train)

In [165]:
tfidf_matrix

TfidfVectorizer(analyzer=<function TfidfEmbeddingVectorizer.fit.<locals>.<lambda> at 0x1a3fa59598>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), norm='l2',
        preprocessor=None, smooth_idf=True, stop_words=None,
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [166]:

idf = tfidf.idf_
print (tfidf.get_feature_names(), idf)

AttributeError: 'TfidfEmbeddingVectorizer' object has no attribute 'idf_'

In [105]:
# corpus = [
#     tokenize(df[0][0]),
#     tokenize(df[0][1]),
#     tokenize(df[0][2])
# ]

In [167]:
x_train = tfidf_matrix.toarray()
y_train = []

for i in range(250):
    y_train.append(train[i][1])
y_train = np.array(y_train)

AttributeError: 'TfidfVectorizer' object has no attribute 'toarray'

In [107]:
x_train.shape

(250, 4031)

In [108]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import f1_score

classifier = SVC(kernel = 'rbf')
linearClassifier = SVC(kernel = 'linear')
classifer = GaussianNB()
classifier.fit(x_train, y_train)
linearClassifier.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [109]:

tfidf_matrix = tfidf.transform(test)
x_test = tfidf_matrix.toarray()
y_test = []

for i in range(250,300):
    y_test.append(train[i][1])
y_test = np.array(y_test)

predictions = classifier.predict(x_test)

In [110]:
# x_train.mean()
# predictions
f1_score(y_test, predictions, average='macro')

/Users/fm_index/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.12222222222222223

In [111]:
predictions = linearClassifier.predict(x_test)
print(predictions)
print(x_test.shape)
# f1_score(y_test, predictions, average='macro')

[4 4 4 4 4 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 3 4 4 4 4 4 4 3 4 4
 4 4 3 4 4 4 4 4 4 4 4 4 4]
(50, 4031)
